In [16]:
from importlib import reload
from src import parsing
from src import model_path
from src import models_path
from src import model_goal
from src import models_goal
from src import inception
from src import model_interface
from src import visualization
from src import util
from src import config
reload(parsing)
reload(model_path)
reload(models_path)
reload(model_goal)
reload(models_goal)
reload(inception)
reload(model_interface)
reload(visualization)
reload(util)
reload(config)

<module 'src.config' from '/home/deus/Documents/trajectory-prediction/src/config.py'>

In [2]:
# load training data
file_path = "data/medium_aa.csv"
train_data, eval_data, test_data = parsing.parse_atc_day(file_path, train_ratio=0.9, eval_ratio=0.1)

100%|██████████| 1224/1224 [00:17<00:00, 70.50it/s] 


Dividing 697 human trajectories: 627 training, 70 eval, 0 test
train (225265, 40, 2) float64 (225265, 25) float32 (225265, 40, 2) float64
eval (23955, 40, 2) float64 (23955, 25) float32 (23955, 40, 2) float64


In [8]:
reload(models_goal)

m = model_goal.ModelGoal()
m.train(models_goal.simple_cnn(train_data), train_data, eval_data, batch_size=1024)
m.save('goal_simple_cnn')
m = None

Epoch 1/10
220/220 [==============================] - 1s 3ms/step - loss: 1.7042 - categorical_accuracy: 0.5314 - val_loss: 1.3216 - val_categorical_accuracy: 0.5670
Epoch 2/10
220/220 [==============================] - 1s 2ms/step - loss: 1.1301 - categorical_accuracy: 0.6314 - val_loss: 1.2790 - val_categorical_accuracy: 0.5862
Epoch 3/10
220/220 [==============================] - 1s 2ms/step - loss: 1.1002 - categorical_accuracy: 0.6411 - val_loss: 1.2693 - val_categorical_accuracy: 0.5875
Epoch 4/10
220/220 [==============================] - 1s 2ms/step - loss: 1.0868 - categorical_accuracy: 0.6450 - val_loss: 1.2605 - val_categorical_accuracy: 0.5956
Epoch 5/10
220/220 [==============================] - 1s 2ms/step - loss: 1.0779 - categorical_accuracy: 0.6467 - val_loss: 1.2553 - val_categorical_accuracy: 0.5964
Epoch 6/10
220/220 [==============================] - 1s 2ms/step - loss: 1.0708 - categorical_accuracy: 0.6480 - val_loss: 1.2521 - val_categorical_accuracy: 0.5962
Epoc

INFO:tensorflow:Assets written to: weights/goal_simple_cnn/assets


In [19]:
reload(inception)

m = model_goal.ModelGoal()
input_shape = train_data[0].shape
input_shape = (input_shape[1], input_shape[2])
inception_model = inception.InceptionTime(
    depth=3
).build(
    input_shape=input_shape, number_output_classes=config.GOAL_GRID_SIZE**2
    )

m.train(inception_model, train_data, eval_data, batch_size=1024, epochs=5)
# m.save('goal_simple_cnn')
m = None

Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           [(None, 40, 2)]      0                                            
__________________________________________________________________________________________________
conv1d_105 (Conv1D)             (None, 40, 32)       64          input_12[0][0]                   
__________________________________________________________________________________________________
max_pooling1d_24 (MaxPooling1D) (None, 40, 2)        0           input_12[0][0]                   
__________________________________________________________________________________________________
conv1d_106 (Conv1D)             (None, 40, 32)       40960       conv1d_105[0][0]                 
____________________________________________________________________________________________

In [4]:
m = model_path.ModelPath(uses_goal=False)
m.train(models_path.simple_lstm(train_data), train_data, eval_data)
m.save('path_simple_lstm')
m = None

Epoch 1/10
6895/6895 [==============================] - 24s 3ms/step - loss: 0.0031 - mean_absolute_error: 0.0360 - val_loss: 0.0024 - val_mean_absolute_error: 0.0319
Epoch 2/10
6895/6895 [==============================] - 22s 3ms/step - loss: 0.0028 - mean_absolute_error: 0.0331 - val_loss: 0.0023 - val_mean_absolute_error: 0.0306
Epoch 3/10
6895/6895 [==============================] - 22s 3ms/step - loss: 0.0027 - mean_absolute_error: 0.0323 - val_loss: 0.0022 - val_mean_absolute_error: 0.0289
Epoch 4/10
6895/6895 [==============================] - 22s 3ms/step - loss: 0.0027 - mean_absolute_error: 0.0320 - val_loss: 0.0021 - val_mean_absolute_error: 0.0288
Epoch 5/10
6895/6895 [==============================] - 22s 3ms/step - loss: 0.0026 - mean_absolute_error: 0.0318 - val_loss: 0.0021 - val_mean_absolute_error: 0.0288
Epoch 6/10
6895/6895 [==============================] - 21s 3ms/step - loss: 0.0026 - mean_absolute_error: 0.0316 - val_loss: 0.0021 - val_mean_absolute_error: 0.029

INFO:tensorflow:Assets written to: weights/path_simple_lstm/assets


INFO:tensorflow:Assets written to: weights/path_simple_lstm/assets


In [5]:
m = model_path.ModelPath(uses_goal=True)
m.train(models_path.simple_lstm(train_data), train_data, eval_data, train_goals=train_data[1], eval_goals=eval_data[1])
m.save('path_simple_lstm_goal')
m = None

Epoch 1/10
6895/6895 [==============================] - 22s 3ms/step - loss: 0.0017 - mean_absolute_error: 0.0283 - val_loss: 0.0014 - val_mean_absolute_error: 0.0252
Epoch 2/10
6895/6895 [==============================] - 22s 3ms/step - loss: 0.0014 - mean_absolute_error: 0.0262 - val_loss: 0.0013 - val_mean_absolute_error: 0.0245
Epoch 3/10
6895/6895 [==============================] - 21s 3ms/step - loss: 0.0014 - mean_absolute_error: 0.0255 - val_loss: 0.0013 - val_mean_absolute_error: 0.0240
Epoch 4/10
6895/6895 [==============================] - 21s 3ms/step - loss: 0.0014 - mean_absolute_error: 0.0252 - val_loss: 0.0012 - val_mean_absolute_error: 0.0236
Epoch 5/10
6895/6895 [==============================] - 21s 3ms/step - loss: 0.0013 - mean_absolute_error: 0.0250 - val_loss: 0.0012 - val_mean_absolute_error: 0.0237
Epoch 6/10
6895/6895 [==============================] - 21s 3ms/step - loss: 0.0013 - mean_absolute_error: 0.0248 - val_loss: 0.0012 - val_mean_absolute_error: 0.023

INFO:tensorflow:Assets written to: weights/path_simple_lstm_goal/assets


INFO:tensorflow:Assets written to: weights/path_simple_lstm_goal/assets
